In [1]:
import pandas as pd
import json
from decimal import Decimal
import uuid
import random
import numpy as np
import boto3
from tqdm import tqdm

In [6]:
def random_date_generator():
    days_to_add = np.arange(0, 365)
    random_date = np.datetime64('2019-01-01') + np.random.choice(365)
    return random_date.item().strftime('%-m/%-d/%Y')

In [10]:
clientkinesis = boto3.client('kinesis',region_name='us-east-1')
kdsname='test'

aws_services = ["Amazon DynamoDB", "Amazon Simple Notification Service", "Amazon Simple Queue Service"
                "Amazon MQ", "Amazon EMR", "AWS Data Pipeline", "Amazon Athena", "AWS Step Functions"]
subject_list = ["FO-CAT", "TH-", "RE-"]
title = ["Tagging tables", "On-demand and transactions", "Some Question about logging"]
message =  ["DynamoDB on-demand and transactions now available in the AWS GovCloud (US) Regions",
            "DynamoDB now supports tagging tables when you create them in the AWS GovCloud (US) Regions",
            "Testing the monitoring and alarm features as you approach your account limits in DynamoDB",
            "Ability to delete a global secondary index before its creation is completed will be very useful",
            "I really love the support for 25 unique items and 4 MB of data per transactional request",
            "Ability to delete a global secondary index before its creation is completed will be very useful"]
email = ["carlos@example.com", "nikhil@example.com", "richard@example.com", "mary@example.com", "marcosdipalma@yahoo.com"]

In [11]:
def generate_data():
    service = np.random.choice(aws_services)
    subject = np.random.choice(subject_list, p=[0.2, 0.4, 0.4])
    new_dict = {"ForumName": service}

    if subject == "FO-CAT":
        new_dict["Subject"] = subject + "-" + str(uuid.uuid4())
        new_dict["Category"] = "Amazon Web Services"
        new_dict["Threads"] = np.random.randint(1, 60)
        new_dict["Messages"] = np.random.randint(15, 95)
        new_dict["Views"] = np.random.randint(80, 1005)
    elif subject == "TH-":
        new_dict["Title"] = np.random.choice(title)
        new_dict["Subject"] = subject + str(uuid.uuid4())
        new_dict["Message"] = np.random.choice(message)
        new_dict["LastPostedBy"] = np.random.choice(email)
        new_dict["Replies"] = np.random.randint(1, 30)
        new_dict["Views"] = np.random.randint(20, 350)
        new_dict["CreatedDate"] = random_date_generator()
    else:
        new_dict["Subject"] = subject + str(uuid.uuid4())
        new_dict["ReplyDateTime"] = random_date_generator()
        new_dict["Message"] = np.random.choice(message)
        new_dict["PostedBy"] = np.random.choice(email)

    return new_dict

In [13]:
repeat = 1500

for i in tqdm(range(repeat), desc = 'Progress Bar'):
    item = generate_data()
    _id='id' + str(np.random.randint(1665586, 8888888))
    response=clientkinesis.put_record(StreamName=kdsname, Data=json.dumps(item), PartitionKey=_id)
    if not response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print("Error...")
        print(response)

Progress Bar: 100%|██████████| 1500/1500 [03:45<00:00,  6.65it/s]
